<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/NasNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 33.8 MB/s 
     |████████████████████████████████| 182 kB 59.7 MB/s 
     |████████████████████████████████| 162 kB 62.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 162 kB 63.2 MB/s 
     |████████████████████████████████| 158 kB 67.8 MB/s 
     |████████████████████████████████| 157 kB 72.6 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |████████████████████████████████| 157 kB 70.8 MB/s 
     |████████████████████████████████| 157 kB 65.4 MB/s 
     |████████████████████████████████| 157 kB 71.9 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 72.3 MB/s 
     |████████████████████████████████| 156 kB 63.6 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [3]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [4]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [13]:
nasnetMobile = tf.keras.applications.NASNetMobile(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_nasnetMobile=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  nasnetMobile,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
nasnetMobile.trainable=False
model_nasnetMobile.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 flatten_1 (Flatten)         (None, 51744)             0         
                                                                 
 dense_2 (Dense)             (None, 32)                1655840   
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 5,925,721
Trainable params: 1,656,005
Non-trainable params: 4,269,716
_________________________________________________________________


In [14]:
model_nasnetMobile.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [15]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "NASNetMobile_30 epochs",
      notes = "with more epochs"
)

In [16]:
%%time
epochs=15
history = model_nasnetMobile.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/15
18/19 [===========================>..] - ETA: 0s - loss: 2.9817 - accuracy: 0.6059

wandb: Adding directory to artifact (/content/wandb/run-20221019_055207-1b28qnfr/files/model-best)... Done. 0.6s


19/19 [==============================] - 103s 5s/step - loss: 2.9144 - accuracy: 0.6098 - val_loss: 0.7733 - val_accuracy: 0.7297
Epoch 2/15
18/19 [===========================>..] - ETA: 0s - loss: 0.4127 - accuracy: 0.8212

wandb: Adding directory to artifact (/content/wandb/run-20221019_055207-1b28qnfr/files/model-best)... Done. 0.4s


19/19 [==============================] - 88s 5s/step - loss: 0.4083 - accuracy: 0.8209 - val_loss: 0.4642 - val_accuracy: 0.8446
Epoch 3/15
18/19 [===========================>..] - ETA: 0s - loss: 0.2106 - accuracy: 0.9080

...

19/19 [==============================] - 90s 5s/step - loss: 0.0324 - accuracy: 0.9932 - val_loss: 0.1511 - val_accuracy: 0.9662
Epoch 6/15
19/19 [==============================] - ETA: 0s - loss: 0.0094 - accuracy: 0.9983

wandb: Adding directory to artifact (/content/wandb/run-20221019_055207-1b28qnfr/files/model-best)... Done. 0.6s


19/19 [==============================] - 89s 5s/step - loss: 0.0094 - accuracy: 0.9983 - val_loss: 0.1456 - val_accuracy: 0.9730
Epoch 7/15
18/19 [===========================>..] - ETA: 0s - loss: 0.0050 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221019_055207-1b28qnfr/files/model-best)... Done. 0.6s


19/19 [==============================] - 89s 5s/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.1362 - val_accuracy: 0.9730
Epoch 8/15
19/19 [==============================] - 3s 125ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1375 - val_accuracy: 0.9730
Epoch 9/15
19/19 [==============================] - 3s 124ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.1423 - val_accuracy: 0.9730
Epoch 10/15
19/19 [==============================] - 3s 130ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.1432 - val_accuracy: 0.9730
Epoch 11/15
19/19 [==============================] - 3s 124ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1442 - val_accuracy: 0.9730
Epoch 12/15
19/19 [==============================] - 3s 124ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.1448 - val_accuracy: 0.9730
Epoch 13/15
19/19 [==============================] - 3s 128ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.1480 - val_accuracy: 0.9730
Epoch 14/15
19/

In [9]:
wandb.finish()

accuracy,▁▄▅▇▇▇█████████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▆▇▆███▇█▇▇▇█
val_loss,█▇▃▂▁▃▂▂▁▂▂▁▁▁▁
accuracy,0.97804
best_epoch,4
best_val_loss,0.35769
epoch,14
loss,0.17155
val_accuracy,0.96622
